In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Set up environment

In [1]:
import torch
from IPython.display import Image, clear_output  # to display images
import shutil
import os
import pandas as pd
import numpy as np
from PIL import Image
import math
import cv2
from collections import deque
from google.colab.patches import cv2_imshow

Clone repo, install dependencies, %cd into ./yolov5 folder and check GPU.

In [ ]:
!git clone https://github.com/ultralytics/yolov5 

In [4]:
cd /content/drive/My Drive/yolo5-localization/yolov5

/content/drive/My Drive/yolo5-localization/yolov5


In [5]:
!pip install -r requirements.txt  # install dependencies
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.5.1+cu101 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


# Prepare data

The directory of the dataset include:

    data/
        images/
              train/
              valid/
        labels/
              train/
              valid/

data/image/train: Including entire images train 

data/image/valid: Including entire images valid 

data/labels/train: Include the entire label corresponding to the name of the image in the folder image/train

data/labels/valid: Include the entire label corresponding to the name of the image in the folder image/valid

For each file label include 5 parameter: "label center_x center_y w h"

In the final set, I adjusted the parameter in my_setting.yaml. It includes some parameter:
  train: the path of the train set
  test: the path of the test set
  nc : number class
  names: an array consisting of the name of the class corresponding to the class id.  
  

# Setting Config 

In my tutorial, I use yolov5s so that, I config yolov5s.yaml, And I only change "nc"m which is number class in my dataset.

Change some parameter in the folder data/my_setting:

    valid: The directory of images/valid
    train: The directory of images/train
    nc : Number class of dataset.




# Train

This repository uses PyTorch framework, and the train.py is the main program, which controls all parameter ( focal loss,weight_decay, momentum, weight_decay, learning rate, ...) and preprocessing (resize size, shear, scale, anchor...)  of the algorithm.

And Now, I runing program with train.py and set some parameter example: 

    --img : resize image to 640x640
    --batch: batch-size 16
    --epochs: 200
    --data: path of the setting file.
    --models: choose models, yolov5 have some model such as yolov5s ( pretrain with small dataset ) , yolov5l (yolov5 pretrain with large dataset)...
    --weight: link weigth for training. The first train model, you might set ''. After that, You can use push link pretrain model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 30 --data data/my_setting.yaml --cfg models/yolov5s_sen.yaml --weights '' --name  CASHEW

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 8 (delta 5), reused 5 (delta 5), pack-reused 3
Unpacking objects: 100% (8/8), done.
From https://github.com/ultralytics/yolov5
   f796708..a040500  master     -> origin/master
Your branch is behind 'origin/master' by 205 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='models/yolov5s_sen.yaml', data='data/my_setting.yaml', device='', epochs=30, evolve=False, img_size=[640], multi_scale=False, name='CASHEW', noautoanchor=False

If you want to continue training, change the weight parameter --weights. In my notebook, I set --weights = weights/last_CASHEW.pt, which I training previous step.

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data data/my_setting.yaml --cfg models/yolov5s_sen.yaml --weights weights/last_CASHEW.pt --name  CASHEW

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Your branch is behind 'origin/master' by 205 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='models/yolov5s_sen.yaml', data='data/my_setting.yaml', device='', epochs=50, evolve=False, img_size=[640], multi_scale=False, name='CASHEW', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='weights/last_CASHEW.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

2020-07-15 11:41:16.744394: I tensorflow/stream_executor/platform/default/dso_load